In [1]:
import numpy as np
import pandas as pd




In [8]:
df = pd.read_csv('../bot/data/training_data.csv')
df.dropna(inplace=True)
cols = ["timestamp","price","demand","pv_power"]
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72224 entries, 227358 to 299586
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  72224 non-null  object 
 1   price      72224 non-null  float64
 2   demand     72224 non-null  float64
 3   pv_power   72224 non-null  float64
dtypes: float64(3), object(1)
memory usage: 2.8+ MB


In [13]:
def create_features(df):
    df['return'] = df['price'].pct_change()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.day_of_week
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['sin_day'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['cos_day'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    # Define peak hours
    df['is_peak'] = df['hour'].apply(lambda x: 1 if 17 <= x < 21 else 0)
    df['is_off_peak'] = df['hour'].apply(lambda x: 1 if 0 <= x < 17 or 21 <= x < 24 else 0)
    df['price_negative'] = (df['price'] < 0).astype(int)
    return df

create_features(df).head()


,timestamp,price,demand,pv_power,return,hour,day_of_week,sin_hour,cos_hour,sin_day,cos_day,is_peak,is_off_peak,price_negative
227358,2023-03-01 00:30:00+00:00,-28.36397,1012.24,2.916,NaN,0,2,0.0,1.0,0.974928,-0.222521,0,1,1
227359,2023-03-01 00:35:00+00:00,-11.00902,1024.27,2.916,-0.611866,0,2,0.0,1.0,0.974928,-0.222521,0,1,1
227360,2023-03-01 00:40:00+00:00,-8.00477,1000.61,2.916,-0.272890,0,2,0.0,1.0,0.974928,-0.222521,0,1,1
227361,2023-03-01 00:45:00+00:00,-8.16710,1036.94,2.916,0.020279,0,2,0.0,1.0,0.974928,-0.222521,0,1,1
227362,2023-03-01 00:50:00+00:00,-32.14319,983.33,2.916,2.935692,0,2,0.0,1.0,0.974928,-0.222521,0,1,1


In [ ]:
def calculate_combined_logic_reward(action, external_state, internal_state):
    reward = 0
    solar_kW_to_battery,charge_kW = action
    price = external_state['price']  
    pv_power = external_state['pv_power']  
    battery_soc = internal_state['battery_soc'] / internal_state['battery_capacity']  # Normalize SOC
    is_peak_hour = external_state['hour'] in [17, 18, 19, 20]  # Define peak hours
    is_off_peak_hour = not is_peak_hour

    # Assuming charge_kW is positive for charging and negative for discharging
    energy_amount = abs(charge_kW)  # Use the absolute value for calculations

    # Charging logic
    if charge_kW > 0:
        if is_off_peak_hour:
            cost = energy_amount * price * 1.05  # 5% penalty for charging during off-peak hours
        else:
            cost = energy_amount * price * 1.40  # Higher cost during peak hours
        reward -= cost
        if solar_kW_to_battery > energy_amount:
            reward += 0.1 * cost  # Bonus for utilizing solar power over grid power

    # Discharging logic
    elif charge_kW < 0:  # Negative charge_kW indicates discharging
        if is_peak_hour:
            revenue = energy_amount * price * 1.30  # Bonus for discharging during peak hours
        elif is_off_peak_hour:
            revenue = energy_amount * price * 0.85  # Reduced revenue for discharging during off-peak hours
        else:
            revenue = energy_amount * price
        reward += revenue

    # Battery efficiency and SOC considerations
    if battery_soc > 0.9:  # Discourage reaching full capacity to avoid overcharging
        reward -= 10  
    elif battery_soc < 0.1:  # Discourage depleting the battery to avoid damage
        reward -= 10  

    return reward